# Mining the Marathon

CISC 4631 Final Project

Paulina Delabra Serrano, Jane Warren, and Donald Ye

Due Dec. 5, 2025

---

**Preprocessing**

This notebook:
- labels the partipant type and drops the wheelchair and handcycle participants
-  drops the splitName and changes the format of the datatset
- converts all the times into seconds
- pivots the time and pace cols per runner then merges to create one row per runner with all **times**
- saves the cleaned csv for data mining




In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# 2025 one hot encoded bib numbers
wheelchair_bibs = [201, 202, 203, 205, 206, 250, 212, 204, 207, 208, 213, 209, 214, 251, 254, 252, 211, 217, 215, 255, 260, 253, 262, 261, 258, 210, 216, 265, 263, 257, 266, 278, 267, 268, 269, 272, 270, 276, 274, 291]
handcycle_bibs = [300, 309, 334, 329, 301, 316, 306, 307, 325, 321, 304, 322, 318, 341, 311, 350, 324, 315, 226, 327, 332, 317, 338, 337, 314, 1252, 386, 378, 308, 303, 310, 312, 397, 340, 385, 320, 328, 394, 384, 333, 395, 39, 388, 398 , 383, 387, 391, 377, 319, 396, 393, 389]

df = pd.read_csv('/content/drive/MyDrive/25-26/fall 2025/CISC 4631 - data mining/final project/nyrr_marathon_2025_summary_56480_runners_WITH_SPLITS.csv')

# label the participant types
df['participant_type'] = 'Runner'

df.loc[df['Bib'].isin(wheelchair_bibs), 'participant_type'] = 'Wheelchair'
df.loc[df['Bib'].isin(handcycle_bibs), 'participant_type'] = 'Handcycle'

df.to_csv('/content/drive/MyDrive/25-26/fall 2025/CISC 4631 - data mining/final project/nyrr_marathon_2025_summary_56480_runners_WITH_SPLITS.csv', index=False)

# check
print(df['participant_type'].value_counts())

participant_type
Runner        1878889
Handcycle        1565
Wheelchair       1337
Name: count, dtype: int64


In [ ]:
df.head(10)

,splitCode,splitName,time,pace,speed,distance,RunnerName,RunnerID,OverallTime,OverallPlace,Gender,Age,City,Country,Bib,participant_type
0,3M,NaN,0:14:43,04:55,12.2,3.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
1,5K,NaN,0:15:13,04:54,12.3,3.11,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
2,4M,NaN,0:19:43,04:56,12.2,4.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
3,5M,NaN,0:24:37,04:56,12.2,5.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
4,6M,NaN,0:29:21,04:54,12.3,6.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
5,10K,NaN,0:30:25,04:54,12.3,6.21,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
6,7M,NaN,0:34:22,04:55,12.2,7.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
7,8M,NaN,0:39:25,04:56,12.2,8.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
8,9M,NaN,0:44:28,04:57,12.1,9.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner
9,15K,NaN,0:46:01,04:57,12.2,9.32,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner


In [ ]:
# drop non runners
df_runners = df[df['participant_type'] == 'Runner'].copy()
print(f"Runners remaining: {len(df_runners):,}")

Runners remaining: 1,878,889


In [ ]:
# convert time into seconds
def convert_pace_to_seconds(pace_str):
    if pd.isna(pace_str):
        return None
    try:
        parts = str(pace_str).split(':')
        if len(parts) == 3: # format HH:MM:SS (e.g., overall time)
            h, m, s = map(int, parts)
            return h * 3600 + m * 60 + s
        elif len(parts) == 2: # format MM:SS (e.g., split pace)
            m, s = map(int, parts)
            return m * 60 + s
        else:
            return None
    except ValueError:
        return None

df_runners['pace_seconds'] = df_runners['pace'].apply(convert_pace_to_seconds)
df_runners = df_runners.dropna(subset=['pace_seconds'])

In [ ]:
df_runners

,splitCode,splitName,time,pace,speed,distance,RunnerName,RunnerID,OverallTime,OverallPlace,Gender,Age,City,Country,Bib,participant_type,pace_seconds
0,3M,NaN,0:14:43,04:55,12.2,3.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,295
1,5K,NaN,0:15:13,04:54,12.3,3.11,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,294
2,4M,NaN,0:19:43,04:56,12.2,4.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,296
3,5M,NaN,0:24:37,04:56,12.2,5.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,296
4,6M,NaN,0:29:21,04:54,12.3,6.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881786,24M,NaN,4:33:56,11:25,5.3,24.00,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,685
1881787,40K,NaN,4:44:59,11:28,5.2,24.85,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,688
1881788,25M,NaN,4:46:48,11:29,5.2,25.00,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,689
1881789,26M,NaN,4:59:20,11:31,5.2,26.00,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,691


In [ ]:
df_runners['OverallTime_seconds'] = df['OverallTime'].apply(convert_pace_to_seconds)

In [ ]:
df_runners['split_time_seconds'] = df['time'].apply(convert_pace_to_seconds)
df_runners

,splitCode,splitName,time,pace,speed,distance,RunnerName,RunnerID,OverallTime,OverallPlace,Gender,Age,City,Country,Bib,participant_type,pace_seconds,OverallTime_seconds,split_time_seconds
0,3M,NaN,0:14:43,04:55,12.2,3.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,295,7689,883
1,5K,NaN,0:15:13,04:54,12.3,3.11,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,294,7689,913
2,4M,NaN,0:19:43,04:56,12.2,4.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,296,7689,1183
3,5M,NaN,0:24:37,04:56,12.2,5.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,296,7689,1477
4,6M,NaN,0:29:21,04:54,12.3,6.00,Benson Kipruto,48836639,2:08:09,1,M,34,Kapsabet,KEN,5.0,Runner,294,7689,1761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881786,24M,NaN,4:33:56,11:25,5.3,24.00,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,685,18145,16436
1881787,40K,NaN,4:44:59,11:28,5.2,24.85,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,688,18145,17099
1881788,25M,NaN,4:46:48,11:29,5.2,25.00,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,689,18145,17208
1881789,26M,NaN,4:59:20,11:31,5.2,26.00,Vincent Virga,48878914,5:02:25,43052,M,59,Naples,USA,45466.0,Runner,691,18145,17960


In [ ]:
df_runners.drop(columns=['pace','OverallTime','time'], inplace=True)
df_runners = df_runners.rename(columns={'pace_seconds': 'pace', 'OverallTime_seconds': 'OverallTime', 'split_time_seconds' : 'time'})
df_runners

,splitCode,splitName,speed,distance,RunnerName,RunnerID,OverallPlace,Gender,Age,City,Country,Bib,participant_type,pace,OverallTime,time
0,3M,NaN,12.2,3.00,Benson Kipruto,48836639,1,M,34,Kapsabet,KEN,5.0,Runner,295,7689,883
1,5K,NaN,12.3,3.11,Benson Kipruto,48836639,1,M,34,Kapsabet,KEN,5.0,Runner,294,7689,913
2,4M,NaN,12.2,4.00,Benson Kipruto,48836639,1,M,34,Kapsabet,KEN,5.0,Runner,296,7689,1183
3,5M,NaN,12.2,5.00,Benson Kipruto,48836639,1,M,34,Kapsabet,KEN,5.0,Runner,296,7689,1477
4,6M,NaN,12.3,6.00,Benson Kipruto,48836639,1,M,34,Kapsabet,KEN,5.0,Runner,294,7689,1761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881786,24M,NaN,5.3,24.00,Vincent Virga,48878914,43052,M,59,Naples,USA,45466.0,Runner,685,18145,16436
1881787,40K,NaN,5.2,24.85,Vincent Virga,48878914,43052,M,59,Naples,USA,45466.0,Runner,688,18145,17099
1881788,25M,NaN,5.2,25.00,Vincent Virga,48878914,43052,M,59,Naples,USA,45466.0,Runner,689,18145,17208
1881789,26M,NaN,5.2,26.00,Vincent Virga,48878914,43052,M,59,Naples,USA,45466.0,Runner,691,18145,17960


In [ ]:
# create two pivots, one for time and one for pace
pivot_time = df_runners.pivot_table(
    index='RunnerID',
    columns='splitCode',
    values='time',
    aggfunc='first'
)

pivot_time = pivot_time.add_prefix('time_')

pivot_pace = df_runners.pivot_table(
    index='RunnerID',
    columns='splitCode',
    values='pace',
    aggfunc='first'
)
pivot_pace = pivot_pace.add_prefix('pace_')

In [ ]:
# merge them
pivot_all = pd.concat([pivot_time, pivot_pace], axis=1)
pivot_all

splitCode,time_10K,time_10M,time_11M,time_12M,time_13M,time_14M,time_15K,time_15M,time_16M,time_17M,...,pace_40K,pace_4M,pace_5K,pace_5M,pace_6M,pace_7M,pace_8M,pace_9M,pace_HALF,pace_MAR
RunnerID,,,,,,,,,,,,,,,,,,,,,
48819809,2867.0,4738.0,5337.0,5852.0,6429.0,7117.0,4374.0,7903.0,8461.0,9004.0,...,579.0,462.0,463.0,461.0,462.0,463.0,464.0,469.0,498.0,582.0
48819810,3745.0,6155.0,6800.0,7450.0,NaN,8791.0,5719.0,9445.0,10083.0,10725.0,...,633.0,593.0,587.0,598.0,602.0,605.0,611.0,614.0,627.0,633.0
48819811,4553.0,7304.0,8052.0,8828.0,9633.0,10412.0,6815.0,11198.0,11977.0,12714.0,...,766.0,749.0,757.0,742.0,734.0,731.0,732.0,732.0,744.0,764.0
48819812,3770.0,6201.0,6836.0,7511.0,NaN,8873.0,5753.0,9562.0,10203.0,10884.0,...,708.0,605.0,609.0,606.0,606.0,610.0,615.0,617.0,631.0,731.0
48819813,3399.0,5487.0,6048.0,6617.0,NaN,7774.0,5118.0,8472.0,9161.0,9935.0,...,639.0,547.0,546.0,547.0,547.0,548.0,549.0,550.0,553.0,638.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48879026,4086.0,6710.0,7406.0,8320.0,NaN,9732.0,6241.0,10456.0,11169.0,11976.0,...,715.0,660.0,662.0,658.0,659.0,657.0,671.0,671.0,695.0,715.0
48879027,2930.0,4741.0,5229.0,5710.0,6188.0,6680.0,4416.0,7176.0,7654.0,8136.0,...,494.0,468.0,466.0,470.0,472.0,473.0,473.0,474.0,477.0,497.0
48879028,3219.0,5164.0,5690.0,6212.0,NaN,7284.0,4818.0,7892.0,8439.0,9002.0,...,559.0,521.0,523.0,521.0,518.0,517.0,517.0,518.0,519.0,563.0


In [ ]:
# create a mapping so we have the times in the right order
distance_map = (
    df_runners[['splitCode', 'distance']]
    .drop_duplicates()
    .set_index('splitCode')['distance']
    .to_dict()
)

def sort_key(col):
    # column name looks like "time_3M" or "pace_5K"
    split = col.split('_')[1]           # e.g. "3M"
    return distance_map[split]

pivot_all = pivot_all.reindex(
    sorted(pivot_all.columns, key=sort_key),
    axis=1
)

In [ ]:
# join the runner info so we have name, gender, overallplace, age, city, country...

runner_info = df_runners.drop_duplicates(subset='RunnerID').set_index('RunnerID')
final_df = runner_info.join(pivot_all, how='left')
final_df = final_df.rename(columns=lambda x: f"split_{x}" if x in pivot_all.columns else x)
final_df

,splitCode,splitName,speed,distance,RunnerName,OverallPlace,Gender,Age,City,Country,...,split_pace_20M,split_pace_21M,split_pace_35K,split_pace_22M,split_pace_23M,split_pace_24M,split_pace_40K,split_pace_25M,split_pace_26M,split_pace_MAR
RunnerID,,,,,,,,,,,,,,,,,,,,,
48836639,3M,NaN,12.2,3.0,Benson Kipruto,1,M,34,Kapsabet,KEN,...,298.0,298.0,297.0,297.0,297.0,296.0,295.0,295.0,294.0,294.0
48839402,3M,NaN,12.3,3.0,Alexander Mutiso,2,M,29,Ngong,KEN,...,298.0,298.0,297.0,297.0,297.0,296.0,295.0,295.0,294.0,294.0
48823815,3M,NaN,12.2,3.0,Albert Korir,3,M,31,Kapkitony,KEN,...,298.0,298.0,297.0,297.0,297.0,296.0,296.0,296.0,296.0,296.0
48865793,3M,NaN,12.2,3.0,Patrick Dever,4,M,29,Raleigh,USA,...,297.0,298.0,297.0,297.0,297.0,296.0,296.0,296.0,296.0,296.0
48878755,3M,NaN,12.3,3.0,Matthias Kyburz,5,M,35,Zurich,CHE,...,299.0,299.0,299.0,299.0,299.0,299.0,298.0,298.0,298.0,298.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48871456,3M,NaN,6.3,3.0,Marjolaine De Valmigere,29594,W,50,Strasbourg,FRA,...,601.0,601.0,602.0,603.0,603.0,605.0,605.0,605.0,605.0,605.0
48842433,3M,NaN,6.6,3.0,Ellen De Valk,29861,W,42,Sint-Niklaas,BEL,...,564.0,572.0,575.0,578.0,590.0,602.0,607.0,607.0,607.0,607.0
48850961,3M,NaN,6.5,3.0,Andrea Vasiliou,21950,W,42,Waterbury,USA,...,556.0,557.0,557.0,557.0,559.0,561.0,562.0,562.0,561.0,561.0


In [ ]:
# find any runners who had NaN in OverallTime and drop from final_df
final_df.dropna(subset='OverallTime', inplace=True)

In [ ]:
# clean up any columns we don't want
final_df.drop(columns=['splitName'], inplace=True)

In [ ]:
final_df.to_csv('/content/drive/MyDrive/25-26/fall 2025/CISC 4631 - data mining/final project/all_runners_2025.csv')